# Using tool to get information about a country, from wikipedia




## Import libaries and prepare the API credential

In [ ]:
import tempfile
import shutil
import os
import subprocess
import requests

from autogen import ConversableAgent, register_function
from autogen.coding import LocalCommandLineCodeExecutor

with open('openai.credential', 'r') as file:
    key = file.read()
MODEL = 'gpt-4o'

## A Python function to call Wiki API


In [ ]:
def describe_country(country_name:str)->str:
    url = "https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=1&titles="+country_name
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        json_data = response.json()  # Extract JSON content
        page = next(iter(json_data["query"]["pages"].values()))
        return page["extract"]
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")


## Agent 1: tool executor (just to wrap up the function)

In [ ]:

wiki_tool = ConversableAgent(
    "wiki_tool",
    system_message = "You are a helpful assistant. You may decide to use some external tools"
    "when you received the return of the tool, which is normally a text description of a country, try to shorten it, "
    "focusing on the relation with other countries, and the history after 1950",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config=False
)


### Agent 2: Tool driver (caller of the function)

In [ ]:
wiki_tool_driver = ConversableAgent(
    "wiki_tool_driver",
    llm_config = {"config_list": [{"model": MODEL, "api_key": key}]},
    code_execution_config=False
)

### Register the tool to both caller and executor

*Question:* Why we need tool agents to use a tool?

In [ ]:
register_function(
    describe_country,
    caller = wiki_tool_driver,
    executor = wiki_tool,
    description = "get country description from wiki"
)

## Now we start

In [ ]:
wiki_tool.initiate_chat(
    wiki_tool_driver,
    message = "I want to know something about china",
    max_turns = 2
)
